# **Hybrid Search Langchain**

In [1]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [9]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get("API_KEY")

index_name = "hybrid-search-langchain-pinecone"

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

## Initialize the pinecone client
pc = Pinecone(api_key=api_key)

## Create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384, ## Dimension of dense vector
        metric = "dotproduct", ## Sparse value supported only for dot product
        spec = ServerlessSpec(cloud="aws", region="us-east-1")
    )

In [10]:
index = pc.Index(index_name)
index

In [11]:
## Vector Embedding and Spase MAtrix
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [12]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default() ## TFIDF
bm25_encoder

In [14]:
sentences = [
    "In 2023, I visited Barcelona",
    "In 2024, I visited Goa",
    "In 2025, I'll visit Budapest"
]

## Applying TFIDF values on these sentences
bm25_encoder.fit(sentences)

## Store the values to the json file
bm25_encoder.dump("bm25_values.json")

## Load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 2998.79it/s]


In [15]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder=bm25_encoder, index=index)

In [16]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000021AE5076150>, index=<pinecone.data.index.Index object at 0x0000021AE4AA6C30>)

In [17]:
retriever.add_texts([
    "In 2023, I visited Barcelona",
    "In 2024, I visited Goa",
    "In 2025, I'll visit Budapest"
])

100%|██████████| 1/1 [00:04<00:00,  4.26s/it]


In [18]:
retriever.invoke("What city did I visit last?")

[Document(page_content='In 2023, I visited Barcelona'),
 Document(page_content='In 2024, I visited Goa'),
 Document(page_content="In 2025, I'll visit Budapest")]